<a href="https://colab.research.google.com/github/Sahanasd2003/E---Learning-web-application-platform/blob/main/E_Learning_web_application_platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Phase 1 : Core Models


 My code covers:


*  Content abstract model with title, slug, created, updated.

* Course model with overview, owner, students, and modules.

* Module model with course, title, description, order.

* ItemBase abstract model with owner, title, created, updated, module, description, order.

* Text, File, Image models inheriting from ItemBase.

* Slug generation with a signal (pre_save).

In [54]:
from django.db import models
from django.contrib.auth.models import User
from django.utils.text import slugify
from django.db.models.signals import pre_save
from django.dispatch import receiver

class Content(models.Model):
    """
    Abstract base class for common fields shared by content models.
    """
    title = models.CharField(max_length=200)
    slug = models.SlugField(max_length=200, unique=True, blank=True)
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        abstract = True
        ordering = ["-created"]

    def __str__(self):
        return self.title

class Course(Content):
    """
    Represents a course created by a user with enrolled students.
    """
    overview = models.TextField()
    owner = models.ForeignKey(
        User, on_delete=models.CASCADE, related_name="courses_created"
    )
    students = models.ManyToManyField(
        User, related_name="courses_joined", blank=True
    )
    module = models.ManyToManyField(
        'Module', related_name="courses_of_module", blank=True
    )

    class Meta:
        verbose_name_plural = "Courses"
        ordering = ["-created"]
        app_label = "courses"

    def __str__(self):
        return self.title

class Module(models.Model):
    """
    Represents a module belonging to a course.
    """
    course = models.ForeignKey(
        Course, on_delete=models.CASCADE, related_name="modules"
    )
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    order = models.IntegerField(default=0)
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        ordering = ["order"]
        verbose_name_plural = "Modules"
        app_label = "courses"

    def __str__(self):
        return self.title

class ItemBase(models.Model):
    """
    Abstract base class for content items within a module.
    """
    owner = models.ForeignKey(
        User, on_delete=models.CASCADE, related_name="%(class)s_related"
    )
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    module = models.ForeignKey(
        Module, on_delete=models.CASCADE, related_name="%(class)s"
    )
    order = models.IntegerField(default=0)
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        abstract = True
        ordering = ["order"]

    def __str__(self):
        return self.title

class Text(ItemBase):
    """
    Text-based content within a module.
    """
    content = models.TextField()

    class Meta:
        verbose_name_plural = "Texts"
        app_label = "courses"

    def __str__(self):
        return self.title

class File(ItemBase):
    """
    File-based content within a module.
    """
    content = models.FileField(upload_to="files")

    class Meta:
        verbose_name_plural = "Files"
        app_label = "courses"

    def __str__(self):
        return self.title

class Image(ItemBase):
    """
    Image-based content within a module.
    """
    content = models.ImageField(upload_to="images")

    class Meta:
        verbose_name_plural = "Images"
        app_label = "courses"

    def __str__(self):
        return self.title

@receiver(pre_save, sender=Course)
def generate_course_slug(sender, instance, *args, **kwargs):
    """
    Automatically generate a slug from the course title before saving.
    """
    if not instance.slug:
        instance.slug = slugify(instance.title)

/usr/local/lib/python3.11/dist-packages/django/db/models/base.py:366: RuntimeWarning: Model 'courses.course_module' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)


Phase 2 : OrderField custom field code.



In this phase,

I created a custom model field named OrderField, which extends Django's IntegerField. The purpose of this field is to automatically manage the order of objects (such as modules within a course or items within a module).

Key Features:
Automatically assigns the next available order number based on related objects.
Accepts a for_fields argument to specify the fields that define the group within which the ordering should apply (e.g., modules within the same course).
Helps maintain proper sequencing without manually setting the order each time.
This field is particularly useful in models like Module and ItemBase to keep content organized and properly ordered.

In [126]:
from django.db import models
from django.contrib.auth.models import User
from django.utils.text import slugify
from django.db.models.signals import pre_save
from django.dispatch import receiver


class OrderField(models.IntegerField):
    """
    Custom model field for ordering objects within a specific context.
    Automatically sets the field's value to the last order value + 1
    if the field is being created and its value is not already set.
    """

    def __init__(self, for_fields=None, *args, **kwargs):
        """
        Initializes the OrderField.

        Args:
            for_fields (list, optional): A list of fields to consider when
                determining the order. Defaults to None.  This will look for the higher order for those fields.
        """
        self.for_fields = for_fields
        super().__init__(*args, **kwargs)

    def pre_save(self, model_instance, add):
        """
        Overrides the pre_save method to automatically set the field's value
        if it's not already set.

        Args:
            model_instance: The instance of the model being saved.
            add (bool): True if the model is being created, False if it's
                being updated.

        Returns:
            int: The value of the OrderField.
        """

        if getattr(model_instance, self.attname) is None:

            qs = self.model.objects.all()


            if self.for_fields:
                filter_conditions = {
                    field: getattr(model_instance, field)
                    for field in self.for_fields
                }
                qs = qs.filter(**filter_conditions)


            last_item = qs.order_by(f'-{self.attname}').first()


            next_order = last_item.order + 1 if last_item else 0


            setattr(model_instance, self.attname, next_order)
            return next_order
        else:
            return super().pre_save(model_instance, add)



class Content(models.Model):
    """
    Abstract base class for common fields shared by content models.
    """
    title = models.CharField(max_length=200)
    slug = models.SlugField(max_length=200, unique=True, blank=True)
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        abstract = True
        ordering = ["-created"]

    def __str__(self):
        return self.title


class Course(Content):
    """
    Represents a course created by a user with enrolled students.
    """
    overview = models.TextField()
    owner = models.ForeignKey(
        User, on_delete=models.CASCADE, related_name="courses_created"
    )
    students = models.ManyToManyField(
        User, related_name="courses_joined", blank=True
    )

    class Meta:
        verbose_name_plural = "Courses"
        ordering = ["-created"]
        app_label = "courses"

    def __str__(self):
        return self.title

    def save(self, *args, **kwargs):

        if not self.slug or self._state.adding or self.title != self._original_title:
            self.slug = slugify(self.title)
        super().save(*args, **kwargs)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self._original_title = self.title


class Module(models.Model):
    """
    Represents a module belonging to a course.
    """
    course = models.ForeignKey(
        Course, on_delete=models.CASCADE, related_name="modules"
    )
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    order = models.IntegerField(default=0)
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        ordering = ["order"]
        verbose_name_plural = "Modules"
        app_label = "courses"

    def __str__(self):
        return self.title


class ItemBase(models.Model):
    """
    Abstract base class for content items within a module.
    """
    owner = models.ForeignKey(
        User, on_delete=models.CASCADE, related_name="%(class)s_related"
    )
    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)
    module = models.ForeignKey(
        Module, on_delete=models.CASCADE, related_name="%(class)s"
    )
    order = OrderField(blank=True, for_fields=['module'])
    created = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now=True)

    class Meta:
        abstract = True
        ordering = ["order"]

    def __str__(self):
        return self.title


class Text(ItemBase):
    """
    Text-based content within a module.
    """
    content = models.TextField()

    class Meta:
        verbose_name_plural = "Texts"
        app_label = "courses"

    def __str__(self):
        return self.title


class File(ItemBase):
    """
    File-based content within a module.
    """
    content = models.FileField(upload_to="files")

    class Meta:
        verbose_name_plural = "Files"
        app_label = "courses"

    def __str__(self):
        return self.title


class Image(ItemBase):
    """
    Image-based content within a module.
    """
    content = models.ImageField(upload_to="images")

    class Meta:
        verbose_name_plural = "Images"
        app_label = "courses"

    def __str__(self):
        return self.title


@receiver(pre_save, sender=Course)
def generate_course_slug(sender, instance, *args, **kwargs):
    """
    Automatically generate a slug from the course title before saving.
    """
    if not instance.slug:
        instance.slug = slugify(instance.title)

Phase 3: Views Interaction Flow


Flow Explanation:
User Browses Courses:

When a user visits the course listing page (CourseListView), they see a paginated list of available courses.
Each course links to its detail page for more information.
User Views Course Details:

On selecting a course, the CourseDetailView displays full course information, including the overview, instructor, enrolled students, and list of modules.
Instructor Manages Modules:

If the logged-in user is the owner (instructor) of the course, they can access the CourseModuleUpdateView.
This view provides a formset where the instructor can add, update, or delete modules directly from one page.
After submitting the formset, the modules are saved and linked to the course.






In [108]:
from django.shortcuts import render, get_object_or_404, redirect
from django.urls import reverse
from django.views import View
from django.contrib.auth.mixins import LoginRequiredMixin
from django.contrib import messages

class ContentCreateUpdateView(LoginRequiredMixin, View):
    module = None
    model = None
    obj = None
    template_name = "courses/manage/content_form.html"
    form_class = None

    def get_model(self, model_name):
        if model_name == 'text':
            return Text
        elif model_name == 'file':
            return File
        elif model_name == 'image':
            return Image
        return None

    def get_form_class(self, model):
        if model == Text:
            return TextForm
        elif model == File:
            return FileForm
        elif model == Image:
            return ImageForm
        return None

    def dispatch(self, request, module_id, model_name, id=None):
        self.module = get_object_or_404(Module, id=module_id, course__owner=request.user)
        self.model = self.get_model(model_name)
        if id:
            self.obj = get_object_or_404(self.model, id=id, owner=request.user, module=self.module)

        self.form_class = self.get_form_class(self.model)

        return super().dispatch(request, module_id, model_name, id)

    def get(self, request, module_id, model_name, id=None):
        form = self.form_class(instance=self.obj)
        return render(request, self.template_name, {'module': self.module, 'form': form, 'obj': self.obj})

    def post(self, request, module_id, model_name, id=None):
        form = self.form_class(instance=self.obj, data=request.POST, files=request.FILES)
        if form.is_valid():
            obj = form.save(commit=False)
            obj.owner = request.user
            obj.module = self.module
            obj.save()

            messages.success(request, 'Content updated successfully.')
            return redirect(reverse('courses:course_detail', kwargs={'slug': self.module.course.slug}))
        else:
            messages.error(request, 'There was an error in the form. Please correct it.')
            return render(request, self.template_name, {'module': self.module, 'form': form, 'obj': self.obj})

Phase 4 :  Forms for Module, Text, File, Image


In this phase, I create Django forms to handle the creation and update of course modules and their related content (such as text, files, and images). Using ModelForm, we can easily link forms to models like Module, Text, File, and Image, enabling efficient data validation and form rendering in templates.

This ensures that course owners can manage their course content dynamically through clean, reusable forms.

In [64]:
from django import forms

class ModuleForm(forms.ModelForm):
    """
    Form to create or update a Module.
    Allows setting the module's title and description.
    """
    class Meta:
        model = Module
        fields = ['title', 'description']

class TextForm(forms.ModelForm):
    """
    Form to add text content to a module.
    You can provide a title, description, and the main text content.
    """
    class Meta:
        model = Text
        fields = ['title', 'description', 'content']

class FileForm(forms.ModelForm):
    """
    Form to upload a file to a module.
    You can add a title, description, and choose the file to upload.
    """
    class Meta:
        model = File
        fields = ['title', 'description', 'content']

class ImageForm(forms.ModelForm):
    """
    Form to upload an image to a module.
    You can add a title, description, and upload the image file.
    """
    class Meta:
        model = Image
        fields = ['title', 'description', 'content']

Phase 5 : URLs

In this phase, I configure the URL patterns to connect our views to specific routes. This allows users to navigate to different parts of the course system (such as listing courses, viewing details, or managing modules). Defining clear URL paths is essential for structuring the app and providing a smooth navigation experience.



In [129]:
from django.shortcuts import render, get_object_or_404, redirect
from django.views import View
from django.views.generic import ListView, DetailView, UpdateView
from django.contrib.auth.mixins import LoginRequiredMixin
from django.contrib import messages
from django.urls import reverse

class CourseListView(ListView):
    model = Course
    template_name = 'courses/course_list.html'
    context_object_name = 'courses'

class CourseDetailView(DetailView):
    model = Course
    template_name = 'courses/course_detail.html'
    context_object_name = 'course'

class CourseModuleUpdateView(LoginRequiredMixin, UpdateView):
    model = Course
    template_name = 'courses/manage/module_form.html'
    form_class = ModuleForm

    def get_queryset(self):
        return self.model.objects.filter(owner=self.request.user)

class ContentCreateUpdateView(LoginRequiredMixin, View):
    module = None
    model = None
    obj = None
    template_name = "courses/manage/content_form.html"
    form_class = None

    def get_model(self, model_name):
        if model_name == 'text':
            return Text
        elif model_name == 'file':
            return File
        elif model_name == 'image':
            return Image
        return None

    def get_form_class(self, model):
        if model == Text:
            return TextForm
        elif model == File:
            return FileForm
        elif model == Image:
            return ImageForm
        return None

    def dispatch(self, request, module_id, model_name, id=None):
        self.module = get_object_or_404(Module, id=module_id, course__owner=request.user)
        self.model = self.get_model(model_name)
        if id:
            self.obj = get_object_or_404(self.model, id=id, owner=request.user, module=self.module)

        self.form_class = self.get_form_class(self.model)

        return super().dispatch(request, module_id, model_name, id)

    def get(self, request, module_id, model_name, id=None):
        form = self.form_class(instance=self.obj)
        return render(request, self.template_name, {'module': self.module, 'form': form, 'obj': self.obj})

    def post(self, request, module_id, model_name, id=None):
        form = self.form_class(instance=self.obj, data=request.POST, files=request.FILES)
        if form.is_valid():
            obj = form.save(commit=False)
            obj.owner = request.user
            obj.module = self.module
            obj.save()

            messages.success(request, 'Content saved successfully.')
            return redirect(reverse('courses:course_detail', kwargs={'slug': self.module.course.slug}))
        else:
            messages.error(request, 'Please correct the errors below.')
            return render(request, self.template_name, {'module': self.module, 'form': form, 'obj': self.obj})


Phase 6 :  Templates Setup

Templates in Django are HTML files where dynamic content is rendered. For the courses app, I have created templates to display:

The Course List.

The Course Detail.

The Module Update view.

In [99]:
import os


templates = {
    "course_list.html": """{% extends 'base.html' %}

{% block content %}
    <div class="container">  <!-- Added container for better layout -->
        <h1>Courses</h1>

        {% if messages %}
            <ul class="messages">
                {% for message in messages %}
                    <li class="{{ message.tags }}">{{ message }}</li>
                {% endfor %}
            </ul>
        {% endif %}

        {% if courses %}
            <ul class="course-list">
                {% for course in courses %}
                    <li class="course-item">
                        <a href="{% url 'courses:course_detail' course.slug %}" class="course-link">{{ course.title }}</a>  <!-- Using slug! -->
                    </li>
                {% endfor %}
            </ul>

            {% if is_paginated %}
                <div class="pagination">
                    {% if page_obj.has_previous %}
                        <a href="?page={{ page_obj.previous_page_number }}">Previous</a>
                    {% endif %}

                    <span>Page {{ page_obj.number }} of {{ paginator.num_pages }}</span>

                    {% if page_obj.has_next %}
                        <a href="?page={{ page_obj.next_page_number }}">Next</a>
                    {% endif %}
                </div>
            {% endif %}
        {% else %}
            <p>No courses available at the moment. Please check back later.</p>
        {% endif %}
    </div>
{% endblock %}
""",

    "course_detail.html": """{% extends 'base.html' %}

{% block content %}
    <div class="container">
        <h1>{{ course.title }}</h1>
        <p class="course-overview">{{ course.overview }}</p> <!-- Applied CSS class to the 'p' tag -->

        <h2>Modules</h2>
        {% if course.modules.all %}
            <ul class="module-list">
                {% for module in course.modules.all %}
                    <li>
                        <strong>{{ module.title }}</strong>: {{ module.description }}
                    </li>
                {% endfor %}
            </ul>
        {% else %}
            <p>This course currently has no modules.</p>
        {% endif %}

        <a href="{% url 'courses:course_module_update' course.slug %}">Manage Modules</a>  <!-- Using slug! -->
    </div>
{% endblock %}
""",

    "manage/module_formset.html": """{% extends 'base.html' %}

{% block content %}
    <div class="container">
        <h1>Manage Modules for {{ course.title }}</h1>

        <form method="post">
            {% csrf_token %}
            <table class="module-formset">
                {{ formset.management_form }}
                <thead>
                    <tr>
                        <th>Title</th>
                        <th>Description</th>
                        <th>Actions</th>
                    </tr>
                </thead>
                <tbody>
                    {% for form in formset %}
                        <tr>
                            <td>
                                {{ form.title }}
                                {% if form.title.errors %}
                                    <ul class="errorlist">
                                        {% for error in form.title.errors %}
                                            <li>{{ error }}</li>
                                        {% endfor %}
                                    </ul>
                                {% endif %}
                            </td>
                            <td>
                                {{ form.description }}
                                {% if form.description.errors %}
                                    <ul class="errorlist">
                                        {% for error in form.description.errors %}
                                            <li>{{ error }}</li>
                                        {% endfor %}
                                    </ul>
                                {% endif %}
                            </td>
                            <td>
                                {% if form.instance.pk %}
                                    <a href="#" class="delete-module">Delete</a>  <!-- Requires JavaScript -->
                                {% endif %}
                            </td>
                        </tr>
                    {% endfor %}
                </tbody>
            </table>

            {% if formset.non_form_errors %}
                <div class="error-messages">
                    {{ formset.non_form_errors }}
                </div>
            {% endif %}

            <button type="submit">Save Changes</button>
            <a href="{% url 'courses:course_detail' course.slug %}">Cancel</a>  <!-- Using slug! -->
        </form>
    </div>
{% endblock %}
"""
}


for path, content in templates.items():
    full_path = os.path.join("courses/templates/courses", path)
    os.makedirs(os.path.dirname(full_path), exist_ok=True)
    with open(full_path, "w") as file:
        file.write(content)

print("Templates have been created successfully in 'courses/templates/courses/'")

Templates have been created successfully in 'courses/templates/courses/'


Phase 7: Fixtures Example


Fixtures in Django are used to preload data into the database. They’re usually in JSON, XML, or YAML formats and help set up initial data like courses, modules, or users.
In this project , I have  simulated fixture loading directly by creating a fixture file and using Django's loaddata command

In [93]:
[
    {
        "model": "auth.user",
        "pk": 1,
        "fields": {
            "password": "pbkdf2_sha256$600000$SaltHere$HashedPassHere",
            "is_superuser": True,
            "username": "admin",
            "first_name": "Admin",
            "last_name": "User",
            "email": "admin@example.com",
            "is_staff": True,
            "is_active": True,
            "date_joined": "2023-10-27T12:00:00Z",
            "groups": [],
            "user_permissions": []
        }
    },
    {
        "model": "auth.group",
        "pk": 1,
        "fields": {
            "name": "Instructors",
            "permissions": [7, 8, 9]
        }
    },
    {
        "model": "auth.user",
        "pk": 2,
        "fields": {
            "password": "pbkdf2_sha256$600000$AnotherSalt$AnotherHashedPass",
            "is_superuser": False,
            "username": "instructor1",
            "first_name": "Jane",
            "last_name": "Doe",
            "email": "jane.doe@example.com",
            "is_staff": False,
            "is_active": True,
            "date_joined": "2023-10-27T13:00:00Z",
            "groups": [1],
            "user_permissions": []
        }
    },
    {
        "model": "courses.course",
        "pk": 1,
        "fields": {
            "title": "Introduction to Python",
            "slug": "introduction-to-python",
            "overview": "A beginner-friendly introduction to the Python programming language.",
            "owner": 2,
            "created": "2023-10-27T14:00:00Z",
            "updated": "2023-10-27T14:00:00Z"
        }
    },
    {
        "model": "courses.course",
        "pk": 2,
        "fields": {
            "title": "Web Development with Django",
            "slug": "web-development-with-django",
            "overview": "Learn how to build web applications using the Django framework.",
            "owner": 2,
            "created": "2023-10-27T15:00:00Z",
            "updated": "2023-10-27T15:00:00Z"
        }
    },
    {
        "model": "courses.module",
        "pk": 1,
        "fields": {
            "course": 1,
            "title": "Getting Started",
            "description": "Setting up your Python environment.",
            "order": 1,
            "created": "2023-10-27T16:00:00Z",
            "updated": "2023-10-27T16:00:00Z"
        }
    },
    {
        "model": "courses.module",
        "pk": 2,
        "fields": {
            "course": 1,
            "title": "Basic Syntax",
            "description": "Understanding Python syntax.",
            "order": 2,
            "created": "2023-10-27T17:00:00Z",
            "updated": "2023-10-27T17:00:00Z"
        }
    },
    {
        "model": "courses.module",
        "pk": 3,
        "fields": {
            "course": 2,
            "title": "Introduction to Django",
            "description": "Setting up a Django project.",
            "order": 1,
            "created": "2023-10-27T18:00:00Z",
            "updated": "2023-10-27T18:00:00Z"
        }
    },
    {
        "model": "courses.text",
        "pk": 1,
        "fields": {
            "owner": 2,
            "title": "Welcome to Python",
            "description": "An introductory text.",
            "module": 1,
            "order": 1,
            "created": "2023-10-27T19:00:00Z",
            "updated": "2023-10-27T19:00:00Z",
            "content": "This is some introductory content for Python."
        }
    },
    {
        "model": "courses.file",
        "pk": 1,
        "fields": {
            "owner": 2,
            "title": "Python Cheat Sheet",
            "description": "A handy cheat sheet.",
            "module": 2,
            "order": 1,
            "created": "2023-10-27T20:00:00Z",
            "updated": "2023-10-27T20:00:00Z",
            "content": "files/python_cheat_sheet.pdf"
        }
    },
    {
        "model": "courses.image",
        "pk": 1,
        "fields": {
            "owner": 2,
            "title": "Django Logo",
            "description": "The Django project logo.",
            "module": 3,
            "order": 1,
            "created": "2023-10-27T21:00:00Z",
            "updated": "2023-10-27T21:00:00Z",
            "content": "images/django_logo.png"
        }
    }
]


[{'model': 'auth.user',
  'pk': 1,
  'fields': {'password': 'pbkdf2_sha256$600000$SaltHere$HashedPassHere',
   'is_superuser': True,
   'username': 'admin',
   'first_name': 'Admin',
   'last_name': 'User',
   'email': 'admin@example.com',
   'is_staff': True,
   'is_active': True,
   'date_joined': '2023-10-27T12:00:00Z',
   'groups': [],
   'user_permissions': []}},
 {'model': 'auth.group',
  'pk': 1,
  'fields': {'name': 'Instructors', 'permissions': [7, 8, 9]}},
 {'model': 'auth.user',
  'pk': 2,
  'fields': {'password': 'pbkdf2_sha256$600000$AnotherSalt$AnotherHashedPass',
   'is_superuser': False,
   'username': 'instructor1',
   'first_name': 'Jane',
   'last_name': 'Doe',
   'email': 'jane.doe@example.com',
   'is_staff': False,
   'is_active': True,
   'date_joined': '2023-10-27T13:00:00Z',
   'groups': [1],
   'user_permissions': []}},
 {'model': 'courses.course',
  'pk': 1,
  'fields': {'title': 'Introduction to Python',
   'slug': 'introduction-to-python',
   'overview': 'A